# 2.2 Example - Using the Experiment Class to Correlate XAS and Process Data 

    In this example process parameter data [LabView, Mass Spectrometer] will be added to the experiment class and then correlated to the XAS data.

    This example assumes that you have previoudly run '2.0 Example - Experiment Class -  XAS Data Processing' and '2.1 Example - Experiment Class -  Normalized XANES-EXAFS-FT' and have a saved 'pickled' experiment.


### Steps:
    
    1. Import Modules
    2. Define Paths
    3. Open Saved Experiment using 'pickle'
    4. Add Process Data to Experiment 
    5. Correlate Process Parameters to XAS data
    6. Save Experiment with 'pickle'

 

### Things to have ready when attemtping this example:

    1. Directory and filename for the 'pickled' experiment 
    2. Scan name of one spectra in the experiment object
    3. Directory and filename of process streams (mass spec and LabView in this example)

## Step 1. Import Modules

In [1]:
# File Handling
import os
import pickle

# Wrapper for xraylarch (larch)
os.chdir('..')
pwd = os.getcwd()
os.chdir('./catxas')
import general as fcts
import xas as xfcts
import plot as pfcts
import process
import experiment as exp

# Plot Handling
%matplotlib inline

## Step 2. Define Paths

    Note: The sample shown here has mass spec and LabView data.  As more process streams are generated, additional formats can be added. This is just an example.

#### Modify:
    1. MS_filename - path and filename, with extension, to mass spectrometer data
    2. LV_filename - path and filename, with extension, to LabView data
    3. pickle_dir - directory containing the pickled experiment
    4. pickpe_name - filename of the pickled experiment (must include extension '.pickle')
    5. scan_name - a name of one fo the spectra in the experiment, used for interrogation throughout

In [2]:
# Mass Spec File (full path):
MS_filename = os.path.join(pwd, 'sample data\SnO2_TPR_MS.csv') 

# AmP LabView File (full path):
LV_filename = os.path.join(pwd, 'sample data\SnO2_TPR_LV.txt')

# Directory Containing Experiment:
pickle_dir = os.path.join(pwd, 'sample results')

# Name of Experiment (must include extension '.pickle')
pickle_name = 'SnO2_TPR.pickle'

# Scan Naame for looking at parts of the Experiment Object
scan_name = '20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0001'

## Step 3. Open Saved Experiment using 'pickle'

    Pickle - A method to save the experimental class and reopen it elsewhere.

In [3]:
pickle_path = os.path.join(pickle_dir,pickle_name)
pickle_in = open(pickle_path,"rb")
my_exp = pickle.load(pickle_in)
pickle_in.close()

## Step 4. Add Process Data to Experiment 

    Note: Additiona datastreams will need their own functions. Please reach otu with questions.

In [4]:
# Add Mass Spec Dataset
my_exp.import_massspec(MS_filename)

# Add LabView Dataset
my_exp.import_labview(LV_filename)

#### Inspection tools to show what and where things are stored in the experiment object
    1. Dictionary of process data for each process stream uploaded - experiment.process_params 
    2. Dataframe showing one of the process streams - experiment.process_params[key]

In [5]:
# 1.  Display Keys of System

my_exp.process_params.keys()

dict_keys(['MS Data', 'LV Data'])

In [6]:
# 2.  Display mass spec datafram

my_exp.process_params['MS Data']

,H2,He,H2O,28,O2,Ar,CO2,T,Vac
Time,,,,,,,,,
2021-06-14 16:20:01,4.299990e-09,1.805990e-07,0.000000e+00,8.599980e-09,4.299990e-09,0.000004,4.299990e-09,26.6066,0.000002
2021-06-14 16:21:11,-2.687490e-09,1.807280e-07,3.364740e-09,9.406230e-09,7.395980e-09,0.000004,-6.664980e-10,26.8216,0.000002
2021-06-14 16:22:25,4.299990e-10,7.245480e-08,3.009990e-09,2.390790e-07,5.084740e-08,0.000004,2.149990e-10,26.8753,0.000002
2021-06-14 16:23:45,5.019160e-10,1.076100e-07,3.790010e-09,1.096490e-08,2.444970e-09,0.000004,6.912230e-11,26.6603,0.000002
2021-06-14 16:25:09,4.829750e-10,1.792020e-07,4.147460e-09,8.601270e-09,2.168300e-09,0.000004,4.668710e-11,26.4991,0.000002
...,...,...,...,...,...,...,...,...,...
2021-06-14 18:20:21,5.416910e-07,4.799540e-10,4.458250e-09,8.252680e-09,1.907100e-09,0.000004,6.298410e-11,34.4542,0.000002
2021-06-14 18:22:07,5.431960e-07,4.621950e-10,4.339750e-09,8.156480e-09,1.886550e-09,0.000004,7.524980e-11,32.7879,0.000002
2021-06-14 18:23:54,5.356710e-07,4.511440e-10,4.237500e-09,8.210780e-09,1.871380e-09,0.000004,7.626030e-11,31.4441,0.000002


## Step 5. Correlate Process Parameters to XAS data

    This step interpolates values from the parameter datasets onto the XAS spectra time grid.

In [7]:
my_exp.correlate_process_params()

Genearted Spectra Summary
Merged Index for Process: MS Data
Merged Index for Process: LV Data


#### Inspection tools to show correlated data
    
    1. Dataframe of XAS spectra correlated to process data - experiment.summary['XAS Spectra Process Params']


In [8]:
# 1. Correlation Table

my_exp.summary['XAS Spectra Process Params']

,File Name,TOS [s],H2,He,H2O,28,O2,Ar,CO2,T,...,Extra TC3 - Extra TC3 [°C],Extra TC4 - Extra TC4 [°C],Extra TC5 - Extra TC5 [°C],Extra TC6 - Extra TC6 [°C],Extra TC7 - Extra TC7 [°C],Extra TC8 - Extra TC8 [°C],Extra DO 1,Extra DO 2,Extra DO 3,Extra DO 4
Time,,,,,,,,,,,,,,,,,,,,,
2021-06-14 16:33:52,20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0001,0.0,4.929417e-10,1.810452e-07,6.951219e-09,8.664734e-09,2.471646e-09,0.000004,7.719114e-11,26.120772,...,2300.190768,2300.464815,2300.464815,2300.464815,2300.343342,2300.343342,0.0,0.0,0.0,0.0
2021-06-14 16:35:37,20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0002,105.0,4.931230e-10,1.827490e-07,6.926960e-09,8.628440e-09,2.426090e-09,0.000004,7.898010e-11,25.907800,...,2300.228154,2300.499174,2300.499174,2300.499174,2300.378966,2300.378966,0.0,0.0,0.0,0.0
2021-06-14 16:37:23,20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0003,211.0,4.713019e-10,1.821100e-07,6.798056e-09,8.669651e-09,2.461149e-09,0.000004,7.286727e-11,26.067593,...,2300.241883,2300.510800,2300.510800,2300.510800,2300.391108,2300.391108,0.0,0.0,0.0,0.0
2021-06-14 16:39:11,20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0004,319.0,4.056493e-09,1.949101e-07,7.054554e-09,9.396570e-09,1.122279e-08,0.000004,1.175604e-10,26.068597,...,2300.262552,2300.529460,2300.529460,2300.529460,2300.409950,2300.409950,0.0,0.0,0.0,0.0
2021-06-14 16:40:57,20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0005,425.0,1.128710e-08,1.397490e-08,7.368780e-09,8.759550e-09,4.797820e-09,0.000004,9.577160e-11,26.015300,...,2300.294947,2300.559966,2300.559966,2300.559966,2300.440447,2300.440447,0.0,0.0,0.0,0.0
2021-06-14 16:42:42,20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0006,530.0,1.628722e-08,7.763893e-09,7.012374e-09,8.508112e-09,3.144702e-09,0.000004,1.011230e-10,25.964397,...,2300.302634,2300.566300,2300.566300,2300.566300,2300.444244,2300.444244,0.0,0.0,0.0,0.0
2021-06-14 16:44:27,20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0007,635.0,1.677209e-07,6.677977e-09,6.732073e-09,8.423802e-09,2.903965e-09,0.000004,9.577366e-11,26.007782,...,2300.321962,2300.584995,2300.584995,2300.584995,2300.463512,2300.463512,0.0,0.0,0.0,0.0
2021-06-14 16:46:13,20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0008,741.0,5.390002e-07,6.013107e-09,6.438386e-09,8.409555e-09,2.787266e-09,0.000004,8.044948e-11,25.961600,...,2300.348302,2300.611476,2300.611476,2300.611476,2300.491137,2300.491137,0.0,0.0,0.0,0.0
2021-06-14 16:47:59,20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0009,847.0,5.453353e-07,5.441864e-09,6.127758e-09,8.365563e-09,2.705257e-09,0.000004,8.813759e-11,26.992001,...,2300.356767,2300.619987,2300.619987,2300.619987,2300.500101,2300.500101,0.0,0.0,0.0,0.0


## Step 6. Save Experiment with 'pickle' 
    Pickle - A method to save the experimental class and reopen it elsewhere.

In [9]:
pickle_out = open(pickle_path,"wb")
pickle.dump(my_exp, pickle_out)
pickle_out.close()